In [1]:
import pandas as pd

def is_within_bounds(row, bounds):
    # Check each value against the corresponding bound
    for val, (low, high) in zip(row, bounds):
        if not (low <= val <= high):
            return False
    return True

def save_top_inputs(inputs_path, outputs_path, column_name='Energy efficiency', top_n=20, bounds=[]):
    # Load the data from Excel files
    inputs_df = pd.read_csv(inputs_path)
    outputs_df = pd.read_csv(outputs_path)

    # Ensure the column_name exists in the outputs
    if column_name not in outputs_df.columns:
        raise ValueError(f"The column {column_name} does not exist in the outputs file.")

    # Apply bounds to inputs and filter data
    valid_inputs_df = inputs_df[inputs_df.apply(lambda row: is_within_bounds(row, bounds), axis=1)]

    # Check if there are enough valid entries
    if len(valid_inputs_df) < top_n:
        print(f"Not enough valid entries found. Only {len(valid_inputs_df)} valid entries available.")
        return "Not enough valid data."

    # Find indices of valid inputs
    valid_indices = valid_inputs_df.index

    # Select corresponding outputs using valid indices
    valid_outputs_df = outputs_df.loc[valid_indices]

    # Find the top N rows with the smallest values in the specified CAPEX column from the valid outputs
    top_outputs = valid_outputs_df.nlargest(top_n, column_name)

    # Find associated inputs using the index from top outputs
    associated_inputs = inputs_df.loc[top_outputs.index]

    # Save the data to CSV files
    associated_inputs.to_csv('top_inputs.csv', index=False)
    top_outputs[[column_name]].to_csv('top_outputs.csv', index=False)

    return "Files saved successfully, found {} valid outputs.".format(len(top_outputs))






In [2]:
## BOUNDS
# electricity prices - 2018-2023 https://tradingeconomics.com/united-kingdom/electricity-price
elec_nom = 89.2 # GBP/MWh - 02 Jan 2024
elec_nom *= 0.804354 # USD/MWh
elec_nom /= 1000 # USD/kWh

# For the nominal/range of the MeOH reactor size from van Dal 2013
ref_co2_flow = 88e3 # kg/h CO2
co2_flow = 28333.3620500565 # kg/h in simulation
co2_ratio = co2_flow/ref_co2_flow
ref_cat_mass = 44500 # kg catalyst
cat_mass = ref_cat_mass*co2_ratio # kg catalyst in simulation
void = 0.5
density = 1775
meoh_nominal_vol = cat_mass * (1/density) * (1/(1-void)) # m3
bounds = [[2.4, 3.6],       # h2 ratio - +/-20% of stoich
        [5000, 10000],    # meoh pressure - van-dal2013
        [210, 270],       # meoh feed temp - van-dal2013
        [0, 1],           # adiabatic/isothermal meoh
        [0.95, 0.991],       # recycle ratio
        [0.8*meoh_nominal_vol, 1.2*meoh_nominal_vol], # meoh reactor volume - van-dal2013 +/- 20%
        [-float('inf'), float('inf')],       # dme feed temperature - peinado2020    
        [1000, 2000],     # dme reaction pressure - peinado2020
        [0,1],            # feed vapour fraction meoh column
        [0,1],            # feed vapour fraction dme-meoh column
        [57*0.8, 57*1.2], # trays col 1 +/- 20% of nominal
        [-float('inf'), float('inf')], # trays col 2 +/- 20% of nominal
        [44/57*0.8, 44/57*1.2], # relative feed location col 1 +/- 20% of nominal
        [-float('inf'), float('inf')]  # relative feed location col 2 +/- 20% of nominal
        ]




In [3]:
import time

start_time = time.time()


inputs_file_path = 'modified_inputs.csv'
outputs_file_path = 'modified_outputs.csv'

# Usage of the function
result = save_top_inputs(inputs_file_path, outputs_file_path, bounds=bounds)
print(result)

# Your code here
time.sleep(2)  # Example code, replace with your actual code

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Files saved successfully, found 20 valid outputs.
Execution time: 2.07 seconds
